In [15]:
from extract_ner import get_all_ners,get_all_project_ids,get_all_project_experimental
from pdf_analyzer.config_load import load_config
from pdf_analyzer.api import API
from omegaconf import OmegaConf

In [11]:
from transformers import pipeline

pipe = pipeline("token-classification", model="dslim/bert-base-NER")

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
server_config = load_config("config/api/grobid-server-config.yaml")
extract_config = load_config("config/api/api-base-config.yaml")
print("SERVER_CONFIG\n"+OmegaConf.to_yaml(server_config))
print("CLOUD_CONFIG\n"+OmegaConf.to_yaml(extract_config))

base_api = API.BaseAPI(extract_config,server_config)

files = base_api.proccesed_files

SERVER_CONFIG
url:
  protocol: http
  api_domain: yordi111nas.synology.me
  port: 8070

CLOUD_CONFIG
data:
  data_dir: data/PDFs
  format: .pdf
  recursive: true
grobid:
  cache: true
  cache_dir: data/xmls
  operation_key: processFulltextDocument
  format: .grobid.tei.xml
  recursive: true

http://yordi111nas.synology.me:8070/api/isalive
GROBID server is up and running
data/xmls\nlp\Bert.grobid.tei.xml already exist, skipping... (use --force to reprocess pdf input files)
data/xmls\nlp\DistillBERT.grobid.tei.xml already exist, skipping... (use --force to reprocess pdf input files)
data/xmls\nlp\Dont_stop_pretraining.grobid.tei.xml already exist, skipping... (use --force to reprocess pdf input files)
data/xmls\nlp\GPT-3.grobid.tei.xml already exist, skipping... (use --force to reprocess pdf input files)
data/xmls\nlp\LIME.grobid.tei.xml already exist, skipping... (use --force to reprocess pdf input files)
data/xmls\nlp\LoRA.grobid.tei.xml already exist, skipping... (use --force to repro

In [17]:
all_orgs=get_all_ners(files,pipe)
all_orgs

[{'name': 'Al-lenNLP', 'score': 0.67498153, 'type': 'ORG', 'paper_id': 2},
 {'name': 'Office of Naval Research',
  'score': 0.9967265,
  'type': 'ORG',
  'paper_id': 2},
 {'name': 'Google', 'score': 0.99887425, 'type': 'ORG', 'paper_id': 2},
 {'name': 'OpenAI', 'score': 0.9626693, 'type': 'ORG', 'paper_id': 3},
 {'name': 'WebText', 'score': 0.7112076, 'type': 'ORG', 'paper_id': 3},
 {'name': 'TerraSwarm', 'score': 0.99291795, 'type': 'ORG', 'paper_id': 4},
 {'name': 'STARnet', 'score': 0.87451035, 'type': 'ORG', 'paper_id': 4},
 {'name': 'Semiconductor Research Corporation',
  'score': 0.9271019,
  'type': 'ORG',
  'paper_id': 4},
 {'name': 'MARCO', 'score': 0.9814992, 'type': 'ORG', 'paper_id': 4},
 {'name': 'DARPA', 'score': 0.976103, 'type': 'ORG', 'paper_id': 4}]

In [19]:
all_project_ids=get_all_project_ids(files)
all_project_ids

[{'project_id': 'N00014-18-1-2670', 'paper_id': 2},
 {'project_id': 'W911NF-13-1-0246', 'paper_id': 4},
 {'project_id': 'N00014-13-1-0023', 'paper_id': 4}]

In [20]:
all_projects=get_all_project_experimental(files)
all_projects

[{'project_id': 'N00014-18-1-2670', 'name': 'MURI', 'paper_id': 2},
 {'project_id': '#W911NF-13-1-0246', 'name': 'ONR', 'paper_id': 4},
 {'project_id': '#N00014-13-1-0023', 'name': 'ONR', 'paper_id': 4}]